In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels= 8, kernel_size=(3,3), stride=(1,1), padding='same')
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding='same')
        self.fc = nn.Linear(in_features=16*7*7, out_features=num_classes )
            
            
            
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape([x.shape[0], -1])
        x = self.fc(x)
        return x
        
        

In [ ]:
model = CNN()
x= torch.randn(64, 1, 28, 28)
print(model(x).shape)

In [ ]:
#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparams
in_channels= 1
num_classes= 10
num_epochs = 100
batch_size = 64
lr = 0.001

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=True)

In [ ]:
model = CNN(in_channels = in_channels, num_classes = num_classes).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [ ]:
#Train Network
for epoch in range(num_epochs):
    if epoch<50:
        lr = 0.001
    else:
        lr = 0.00001
    for batch_id, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        
        #Forward
        scores = model(data)
        loss = loss_function(scores, targets)
        
        #Backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent
        optimizer.step()
        

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on taining data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            

            
        
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on taining data<br>
Got 59988/60000 with accuracy 99.98<br>
Checking accuracy on test data<br>
Got 9867/10000 with accuracy 98.67